In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "C:/rppg/cohface_datatape_32x72x72_train.h5"
valid_tape = "C:/rppg/cohface_datatape_32x72x72_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_ep(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_ep(train).cache(f'{tmp}/ep'), to_tf_ep(valid).cache(f'{tmp}/ep')
train, valid = to_tf_ep(train), to_tf_ep(valid)

In [3]:
ep = EP(n=32)
ep.compile(optimizer='Nadam', loss='mse')
ep.build(input_shape=(None, 72, 72, 3))
print(f'Flops per frame:{get_flops(ep, input_sig=[tf.TensorSpec([160, 72, 72, 3])])/160:.0f}')
ep.summary()

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Flops per frame:230341628
Model: "ep"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tsm (TSM)                   multiple                  0         
                                                                 
 conv2d (Conv2D)             multiple                  896       
                                                                 
 conv2d_1 (Conv2D)           multiple                  9248      
                                                                 
 conv2d_2 (Conv2D)           multiple                  18496     
                                                                 
 conv2d_3 (Conv2D)           multiple                  36928     
                                                            

In [ ]:
ep.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/EfficientPhys_Cohface
                                                                                                         .h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ep.load_weights('../weights/EfficientPhys_Cohface.h5')

Epoch 1/10
  14818/Unknown - 4007s 270ms/step - loss: 0.3661
Epoch 1: val_loss improved from inf to 0.28196, saving model to ../weights\EfficientPhys_CCNU.h5
14818/14818 [==============================] - 4491s 302ms/step - loss: 0.3661 - val_loss: 0.2820
Epoch 2/10
14818/14818 [==============================] - ETA: 0s - loss: 0.3677
Epoch 2: val_loss improved from 0.28196 to 0.27501, saving model to ../weights\EfficientPhys_CCNU.h5
14818/14818 [==============================] - 4610s 310ms/step - loss: 0.3677 - val_loss: 0.2750
Epoch 3/10
 8863/14818 [================>.............] - ETA: 49:50 - loss: 0.3665

In [3]:
ep.load_weights('../weights/EfficientPhys_Cohface.h5')

In [11]:
eval_on_dataset(test_set_CCNU, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_CCNU.h5')

100%|██████████| 179/179 [05:23<00:00,  1.81s/it]


{'Sliding window': {'MAE': 1.046, 'RMSE': 3.407, 'R': 0.94347},
 'Whole video': {'MAE': 0.533, 'RMSE': 1.237, 'R': 0.99135}}

In [12]:
get_metrics_HRV('../results/EfficientPhys_CCNU_CCNU.h5')

{'SDNN': {'MAE': 32.437, 'RMSE': 42.745, 'R': 0.52057}}

In [7]:
eval_on_dataset(test_set_CCNU_rPPG, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [01:14<00:00,  1.38s/it]


{'Sliding window': {'MAE': 1.003, 'RMSE': 3.392, 'R': 0.93875},
 'Whole video': {'MAE': 0.442, 'RMSE': 0.775, 'R': 0.9965}}

In [8]:
get_metrics_HRV('../results/EfficientPhys_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 43.742, 'RMSE': 53.725, 'R': 0.35571}}

In [5]:
eval_on_dataset(test_set_PURE, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_PURE.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_PURE.h5')

100%|██████████| 59/59 [00:51<00:00,  1.15it/s]


{'Sliding window': {'MAE': 2.848, 'RMSE': 10.329, 'R': 0.90378},
 'Whole video': {'MAE': 1.335, 'RMSE': 5.967, 'R': 0.96764}}

In [9]:
get_metrics_HRV('../results/EfficientPhys_CCNU_PURE.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


{'SDNN': {'MAE': 27.982, 'RMSE': 44.007, 'R': 0.4679}}

In [14]:
eval_on_dataset(test_set_UBFC_rPPG2, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:42<00:00,  1.01s/it]


{'Sliding window': {'MAE': 1.479, 'RMSE': 3.884, 'R': 0.97467},
 'Whole video': {'MAE': 1.027, 'RMSE': 1.448, 'R': 0.99689}}

In [10]:
get_metrics_HRV('../results/EfficientPhys_CCNU_UBFC.h5')

{'SDNN': {'MAE': 10.06, 'RMSE': 15.37, 'R': 0.827}}

In [16]:
eval_on_dataset(test_set_MMPD, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_MMPD.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent'])

100%|██████████| 660/660 [08:56<00:00,  1.23it/s]


{'Sliding window': {'MAE': 1.508, 'RMSE': 5.313, 'R': 0.83583},
 'Whole video': {'MAE': 1.577, 'RMSE': 5.404, 'R': 0.82064}}

In [5]:
eval_on_dataset(test_set_COHFACE, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_COHFACE.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_COHFACE.h5',)

100%|██████████| 164/164 [03:57<00:00,  1.45s/it]


{'Sliding window': {'MAE': 4.48, 'RMSE': 11.905, 'R': 0.54385},
 'Whole video': {'MAE': 3.944, 'RMSE': 12.025, 'R': 0.52752}}